## Pytorch FCN

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np

In [2]:
# 가상 데이터 생성
input_dim = 20
num_classes = 10
X_train = np.random.rand(1000, input_dim).astype(np.float32)
y_train = np.random.randint(num_classes, size=1000).astype(np.int64)
X_test = np.random.rand(200, input_dim).astype(np.float32)
y_test = np.random.randint(num_classes, size=200).astype(np.int64)

# PyTorch 데이터셋 및 데이터로더
train_dataset = TensorDataset(torch.tensor(X_train), torch.tensor(y_train))
test_dataset = TensorDataset(torch.tensor(X_test), torch.tensor(y_test))
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [3]:
class FCNN(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(FCNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, num_classes)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

model = FCNN(input_dim, num_classes)

In [4]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [5]:
num_epochs = 20

for epoch in range(num_epochs):
    model.train()
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

Epoch [1/20], Loss: 2.2795
Epoch [2/20], Loss: 2.3585
Epoch [3/20], Loss: 2.3670
Epoch [4/20], Loss: 2.3156
Epoch [5/20], Loss: 2.3295
Epoch [6/20], Loss: 2.2440
Epoch [7/20], Loss: 2.2808
Epoch [8/20], Loss: 2.2175
Epoch [9/20], Loss: 2.3119
Epoch [10/20], Loss: 2.3069
Epoch [11/20], Loss: 2.2483
Epoch [12/20], Loss: 2.1405
Epoch [13/20], Loss: 2.2819
Epoch [14/20], Loss: 2.1482
Epoch [15/20], Loss: 2.1272
Epoch [16/20], Loss: 2.2794
Epoch [17/20], Loss: 1.9934
Epoch [18/20], Loss: 2.1651
Epoch [19/20], Loss: 2.3674
Epoch [20/20], Loss: 2.2815


In [6]:
# 모델 평가
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        print(outputs.data)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Test Accuracy: {accuracy:.2f}%")

# 모델 예측
sample_input = torch.tensor(X_test[:5])
predictions = model(sample_input)
_, predicted_classes = torch.max(predictions, 1)
print("Predicted classes: ", predicted_classes.numpy())
print("True classes: ", y_test[:5])

tensor([[ 0.1151, -0.2581, -0.0435, -0.4497,  0.1774, -0.2613, -0.3146,  0.1853,
          0.3881, -0.2580],
        [ 0.2112, -0.3568, -0.4284, -0.0115,  0.5388, -0.2462, -0.1922, -0.1507,
         -0.5545, -0.3362],
        [ 0.2410, -0.6752, -0.2640, -0.1397,  0.1085,  0.1336, -0.3577,  0.2828,
          0.1190, -0.4156],
        [ 0.0815, -0.6283, -0.1989,  0.1229,  0.1791,  0.0200, -0.3117,  0.1421,
         -0.0711, -0.4930],
        [-0.0790, -0.4696, -0.2954, -0.2298, -0.0875, -0.1356, -0.3534,  0.1147,
          0.5418, -0.0828],
        [-0.0321, -0.6319,  0.2034, -0.2276,  0.1018, -0.3975, -0.4587, -0.2021,
          0.3199, -0.3132],
        [-0.2413,  0.1734, -0.0157, -0.0845,  0.2766, -0.6116, -0.3845, -0.3810,
         -0.2946,  0.2221],
        [-0.0903, -0.5304, -0.1598, -0.2611, -0.3936,  0.1778, -0.2047,  0.3747,
          0.5458, -0.1984],
        [-0.1869,  0.1200, -0.0700, -0.1982, -0.1537,  0.0354, -0.1504,  0.2060,
         -0.0842, -0.1648],
        [-0.1464, -